Tutorial for finding breaks and plotting paths from the sample GPS data

Imports

In [14]:
import os 
import seaborn as sns
import folium
import numpy as np

# import custom functions
os.chdir('..')
from RuckFinal import DataLoading, Preprocessing
os.chdir(os.getcwd()+r'\Tutorials')

Load data

In [2]:
# Initialize path to data (UTM-converted datasets)
data_dir = os.getcwd() + '\\Data\\csv\\UTM_data'

# Load datasets
raw_datasets = DataLoading.load_data(data_dir)

Preprocess data<br>
(Pivot and interpolate)

In [3]:
# Re-shape datasets
datasets = DataLoading.pivot_datsets(raw_datasets)

# drop outliers and interpolate datasets
interp_datasets = Preprocessing.interpolate_datasets(datasets, threshold = 0.99)

Detect breaks and get 'movement period' slices

In [4]:
# find breaks for the interpolated datsets
rucks_bySquad, rests_bySquad, all_stops = Preprocessing.get_slices_byArea(interp_datasets, plot=False) 

Get one example squad's dataset 

In [ ]:
# Get one example squad's dataset 
data = datasets[1]
stops = all_stops[1]

Plot interactive folium map of individual's paths and group break locations

In [23]:
# initialize map
m = folium.Map(
    # location = [42.49079459046541, -71.66046870896011],
    location = [42.4942, -71.6582],
    zoom_start=13.5,
    tiles='OpenStreetMap',
    width=1024,
    height=768,
    control_scale = True
)

# get names and hard-code colors
snames = stops[0].traj_id.unique()
color_dictionary = dict(zip(snames, sns.color_palette(as_cmap=True)[:len(snames)]))

# loop thorugh names to plot paths
for name in snames:
    # get this soldier
    this_soldier = data[[x for x in data.columns if name in x]] 
    # plot path
    folium.PolyLine(this_soldier[['latitude','longitude']], color=color_dictionary[name], smooth_factor=0, tooltip=name + " path").add_to(m)

# loop through stops to plot them 
for f in stops:
    # get coords from geometry object
    pts_x = np.mean([g for g in f.geometry.x])
    pts_y = np.mean([g for g in f.geometry.y])
    # plot as circle
    folium.Circle(radius=50, location=[pts_y, pts_x] ,color='crimson', fill=True, tooltip='Rest '+ str(f.end_time.max() - f.start_time.min()), popup='Rest '+ str(f.end_time.max() - f.start_time.min())).add_to(m)

# display map
display(m)